In [1]:
import wandb


In [2]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: khachblb06 (khachblb06-polytechnic-of-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
!pip install --upgrade datasets fsspec


In [4]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer

import datasets
from datasets import load_dataset
import torch
import transformers
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModel
import pandas as pd
import numpy as np



## Loading and preprocessing  The IMDB Dataset

In [5]:
dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
dataset['train'][89]

{'text': "or anyone who was praying for the sight of Al Cliver wrestling a naked, 7ft tall black guy into a full nelson, your film has arrived! Film starlet Laura Crawford (Ursula Buchfellner) is kidnapped by a group who demand the ransom of $6 million to be delivered to their island hideaway. What they don't count on is rugged Vietnam vet Peter Weston (Cliver) being hired by a film producer to save the girl. And what they really didn't count on was a local tribe that likes to offer up young women to their monster cannibal god with bloodshot bug eyes.<br /><br />Pretty much the same filming set up as CANNIBALS, this one fares a bit better when it comes to entertainment value, thanks mostly a hilarious dub track and the impossibly goofy monster with the bulging eyes (Franco confirms they were split ping pong balls on the disc's interview). Franco gets a strong EuroCult supporting cast including Gisela Hahn (CONTAMINATION) and Werner Pochath (whose death is one of the most head-scratchin

In [7]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
def preprocess(example):
  return tokenizer(example['text'], truncation = True, padding = 'max_length', max_length = 512,)

In [9]:
encoded_dataset = dataset.map(preprocess, batched = True)
encoded_dataset = encoded_dataset.rename_column('label', 'labels')
encoded_dataset.set_format(type = 'torch', columns = ["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
encoded_dataset['train'][89]

{'labels': tensor(0),
 'input_ids': tensor([     1,    289,   1012,    328,    284,  10481,    270,    262,   3941,
            265,   2513,  27918,    834,  11876,    266,   8775,    261,    574,
           5359,   4180,    937,   1863,    352,    266,    540, 115701,    261,
            290,    933,    303,   2525,    300,   4569,  68192,   7186,  15763,
            287,  34271,    268,   9328,  34576,  38701,   4437,    285,    269,
          20664,    293,    266,    596,    328,   1935,    262,  27925,    265,
            419,    765,    705,    264,    282,   2621,    264,    308,   2623,
          59905,    260,    458,    306,    418,    280,    297,   2795,    277,
            269,  12855,   5681,  10377,   2557,  23746,    287,    711,  56052,
            285,    411,   5458,    293,    266,    933,   4834,    264,   1213,
            262,   1651,    260,    414,    339,    306,    431,    590,    280,
            297,   2795,    277,    284,    266,    588,  10966,    272,  

## Base Model

In [11]:
base_model = AutoModel.from_pretrained("microsoft/deberta-v3-small")


pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

## Model with LoRA


In [12]:
peft_config = LoraConfig(
    task_type = TaskType.SEQ_CLS,
    inference_mode = False,
    r = 8,
    lora_alpha = 32,
    lora_dropout=0.1,
    bias="none"
)
model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-small", num_labels=2)
model_lora = get_peft_model(model, peft_config)
model_lora.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 148,994 || all params: 142,045,444 || trainable%: 0.1049


## Model with only Head


In [13]:
model_classifier = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-small", num_labels=2)
for param in model_classifier.deberta.parameters():
    param.requires_grad = False

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Custom Trainer for computing Metrics and Loss

In [14]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):

        outputs = model(**inputs)
        loss = outputs.loss

        if self.state.global_step % 10 == 0:
            self.log({"train/loss": loss.item()})

        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=1)

    return {
            "eval_f1_weighted": f1_score(labels, preds, average="weighted"),
            "accuracy": (preds == labels).mean(),
        }

# Training and Evaluating Model with only Head

In [15]:
wandb.init(project="deberta-comparison", name="deberta-classifier")

In [16]:
training_args_classifier = TrainingArguments(
    output_dir="./outputs_classifier",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    run_name="deberta-classifier-run",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_weighted",
    greater_is_better=True
)


In [17]:
trainer_classifier = CustomTrainer(
    model=model_classifier,
    args=training_args_classifier,
    train_dataset=encoded_dataset["train"].shuffle(seed=42).select(range(5000)),
    eval_dataset=encoded_dataset["test"].shuffle(seed=42).select(range(1500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-820853687.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer_classifier = CustomTrainer(


In [18]:
trainer_classifier.train()

Epoch,Training Loss,Validation Loss,F1 Weighted,Accuracy
1,0.690500,0.691840,0.324483,0.492000
2,0.689100,0.690190,0.329765,0.493333
3,0.702700,0.689519,0.353480,0.503333


TrainOutput(global_step=939, training_loss=0.692962336717957, metrics={'train_runtime': 504.1498, 'train_samples_per_second': 29.753, 'train_steps_per_second': 1.863, 'total_flos': 1987081758720000.0, 'train_loss': 0.692962336717957, 'epoch': 3.0})

In [19]:
trainer_classifier.evaluate()

{'eval_f1_weighted': 0.3534801803596744,
 'eval_loss': 0.6895185708999634,
 'eval_accuracy': 0.5033333333333333,
 'eval_runtime': 34.4601,
 'eval_samples_per_second': 43.529,
 'eval_steps_per_second': 2.728,
 'epoch': 3.0}

In [20]:
wandb.finish()

eval/accuracy,▁▂██
eval/f1_weighted,▁▂██
eval/loss,█▃▁▁
eval/runtime,▁▃▃█
eval/samples_per_second,█▆▆▁
eval/steps_per_second,█▆▆▁
train/epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇█
train/global_step,▁▁▁▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▃▄▁▆▄▃▄▇▂▃▂▄▄▂▂█▂▃▁▃▄▃▂▁▁▂▁▃▂▂▁▄▁█▂▁▁▃▂▂
train/learning_rate,█████▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁
train/loss,▁█▆▅▂▃▃▄▃▅▅▁▄▅▆▅▃▂▃▅▄▂▃▆▄▂▅▃▄▄▄▃▁▅▂▄▆▁▃▃


## Training and Evaluating Model with LoRA

In [21]:
wandb.init(project="deberta-comparison", name="deberta-lora")

In [22]:
training_args_lora = TrainingArguments(
    output_dir="./outputs_lora",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    run_name="deberta-lora-run",
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_weighted",
    greater_is_better=True
)


In [23]:
trainer_lora = CustomTrainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=encoded_dataset["train"].shuffle(seed=42).select(range(5000)),
    eval_dataset=encoded_dataset["test"].shuffle(seed=42).select(range(1500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-21568634.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer_lora = CustomTrainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
trainer_lora.train()

Epoch,Training Loss,Validation Loss,F1 Weighted,Accuracy
1,0.694200,0.692238,0.324483,0.492000
2,0.664400,0.658745,0.855966,0.856000
3,0.567200,0.562326,0.858651,0.858667


TrainOutput(global_step=939, training_loss=0.6635792826692136, metrics={'train_runtime': 847.756, 'train_samples_per_second': 17.694, 'train_steps_per_second': 1.108, 'total_flos': 1993947402240000.0, 'train_loss': 0.6635792826692136, 'epoch': 3.0})

In [25]:
trainer_lora.evaluate()

{'eval_f1_weighted': 0.8586505787522855,
 'eval_loss': 0.5623260736465454,
 'eval_accuracy': 0.8586666666666667,
 'eval_runtime': 35.133,
 'eval_samples_per_second': 42.695,
 'eval_steps_per_second': 2.676,
 'epoch': 3.0}

In [26]:
wandb.finish()

eval/accuracy,▁███
eval/f1_weighted,▁███
eval/loss,█▆▁▁
eval/runtime,▁█▅▅
eval/samples_per_second,█▁▄▄
eval/steps_per_second,█▁▅▅
train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▆▆▆▆▆▆▆▆▇▇████
train/grad_norm,▂▄▂▄▅▇▂▃▆▁▂▅▂▂▆▂▂▂▁▂▄▃▁▂▅▃▁▂▂▂▂▂▁▂█▂▃▃▂▄
train/learning_rate,█████▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁
train/loss,█▇▇██▇█████▇▇▇█████▇█▇▆▇▆▅▄▄▃▃▃▃▃▂▃▂▂▁▂▁
